In [ ]:
pip install ultralytics cvzone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cach

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone
from google.colab.patches import cv2_imshow



model=YOLO('best.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :
        point = [x, y]
        print(point)



cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)





cap=cv2.VideoCapture('cr.mp4')


my_file = open("coco1.txt", "r")
data = my_file.read()
class_list = data.split("\n")
#print(class_list)



count=0

while True:
    ret,frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue


    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
    results=model.predict(frame)
#    print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")


#    print(px)
    for index,row in px.iterrows():
#        print(row)

        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]

        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)
        cvzone.putTextRect(frame,f'{c}',(x1,y1),1,1)


    cv2_imshow("RGB", frame)
    if cv2.waitKey(0)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

def preprocess(text):
    new_text = []


    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

print("Enter Text")
text = input()
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)


ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Enter Text


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

url= []
search = input("Search: ")
results = 5  # valid options 10, 20, 30, 40, 50, and 100
page = requests.get(f"https://www.google.com/search?q={search}&num={results}")
soup = BeautifulSoup(page.content, "html5lib")
links = soup.findAll("a")
print(links)
for link in links :
    link_href = link.get('href')
    if "url?q=" in link_href and not "webcache" in link_href:
        #print(link.get('href').split("?q=")[1].split("&sa=U")[0])
        url.append(link.get('href').split("?q=")[1].split("&sa=U")[0])

url.pop(0)
url.pop(0)
url.pop(len(url)-1)
url.pop(len(url)-1)
print("\n\n\n",url)

Search: mughal empire
[<a href="/?sa=X&amp;ved=0ahUKEwin2Jq4rYCGAxUCBEQIHeCbBbQQOwgC"><span class="V6gwVd">G</span><span class="iWkuvd">o</span><span class="cDrQ7">o</span><span class="V6gwVd">g</span><span class="ntlR9">l</span><span class="iWkuvd tJ3Myc">e</span></a>, <a href="/search?q=mughal+empire&amp;num=5&amp;sca_esv=1408e98ab1991b60&amp;sca_upv=1&amp;ie=UTF-8&amp;gbv=1&amp;sei=oKQ8ZqeIMYKIkPIP4LeWoAs">here</a>, <a class="eZt8xd" href="/search?q=mughal+empire&amp;num=5&amp;sca_esv=1408e98ab1991b60&amp;sca_upv=1&amp;ie=UTF-8&amp;tbm=isch&amp;source=lnms&amp;sa=X&amp;ved=0ahUKEwin2Jq4rYCGAxUCBEQIHeCbBbQQ_AUIBigB">Images</a>, <a class="eZt8xd" href="/search?q=mughal+empire&amp;num=5&amp;sca_esv=1408e98ab1991b60&amp;sca_upv=1&amp;ie=UTF-8&amp;tbm=nws&amp;source=lnms&amp;sa=X&amp;ved=0ahUKEwin2Jq4rYCGAxUCBEQIHeCbBbQQ_AUIBygC">News</a>, <a class="eZt8xd" href="/search?q=mughal+empire&amp;num=5&amp;sca_esv=1408e98ab1991b60&amp;sca_upv=1&amp;ie=UTF-8&amp;tbm=bks&amp;source=lnms&amp;sa=X

In [ ]:
content=[]
for link in url:
    req=requests.get(link)
    soup= BeautifulSoup(req.content, "html.parser")
    text=soup.get_text()
    tex=text.replace(" ","")
    content.append(text)